In [8]:
import torch
x = torch.rand(1,3,200,210)

#### Depth-wise Convolution 

In [2]:
from torch import nn
class depthwise_conv(nn.Module):
    def __init__(self, nin, kernels_per_layer):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin * kernels_per_layer, kernel_size=3, padding=1, groups=nin)


    def forward(self, x):
        out = self.depthwise(x)
        return out

In [38]:
conv = nn.Conv2d(3,6,kernel_size=3,padding=1,groups=3)

print('weight',conv.weight.shape)

weight torch.Size([6, 1, 3, 3])


In [39]:
conv(x).shape

torch.Size([1, 6, 200, 210])